In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)

# Import CDR Data

In [2]:
df = pd.read_html('cdr.xls')
df=pd.concat(df)
df=df.iloc[:,:-1]

dispos_col= ['#','Campaign','LeadID','DIDNumber','QueueName','Prefix','DialedNumber','CallType','Date/Time',
             'SIP','AgentID','AgentName','Pri/Voip','RingTime','BillSec','TotalTalktime','DialerDisposition',
             'HangupCode','HangupDescription','HangupBy']

df.columns = dispos_col
df.drop(['#','QueueName','Prefix','Pri/Voip'], axis=1, inplace = True)

df['DialedNumber'] = df['DialedNumber'].astype(str)
df['DialedNumber'] = df['DialedNumber'].str.replace(" ()","")
df['DialedNumber'] = df['DialedNumber'].str.slice(2,11)

df['DIDNumber']    = df['DIDNumber'].astype(str)
df['DIDNumber']    = df['DIDNumber'].str.replace(" ()","")
df['DIDNumber']    = df['DIDNumber'].str.slice(2,11)

df['BillSec'] = df['BillSec'].astype(str)
df['BillSec'] = df['BillSec'].str.replace(".","")
df['BillSec'] = df['BillSec'].str.slice(0,8)

df['RingTime'] = df['RingTime'].astype(str)
df['RingTime'] = df['RingTime'].str.replace(".","")
df['RingTime'] = df['RingTime'].str.slice(0,8)

df['TotalTalktime'] = df['TotalTalktime'].astype(str)
df['TotalTalktime'] = df['TotalTalktime'].str.replace(".","")
df['TotalTalktime'] = df['TotalTalktime'].str.slice(0,8)

tt = []
for i in df['TotalTalktime']:
    try:
        i = str(i)
        i = i.replace(":", "")
        h = i[:2]
        m = i[2:4]
        s = i[4:]
        h = int(h)
        m = int(m)
        s = int(s)
        min = (h*60)+(m*1)+(s/60)
        min = round(min,3)
        tt.append(min)
    except:
        tt.append(0)

df['TotalTalktime'] = tt

rt = []
for i in df['RingTime']:
    try:
        i = str(i)
        i = i.replace(":", "")
        h = i[:2]
        m = i[2:4]
        s = i[4:]
        h = int(h)
        m = int(m)
        s = int(s)
        min = (h*60)+(m*1)+(s/60)
        min = round(min,3)
        rt.append(min)
    except:
        rt.append(0)
        

df['RingTime'] = rt  

bs = []
for i in df['BillSec']:
    try:
        i = str(i)
        i = i.replace(":", "")
        h = i[:2]
        m = i[2:4]
        s = i[4:]
        h = int(h)
        m = int(m)
        s = int(s)
        min = (h*60)+(m*1)+(s/60)
        min = round(min,3)
        
        bs.append(min)
    except:
        bs.append(0)
        

df['BillSec'] = bs 

df['Date/Time']= pd.to_datetime(df['Date/Time'])

did = []

for i in df['DIDNumber']:
    try:
        i = int(i)
        
        did.append(i)
    except:
        did.append('')
        

df['DIDNumber'] = did

dn = []

for i in df['DialedNumber']:
    try:
        i = int(i)
        
        dn.append(i)
    except:
        dn.append('')
        

df['DialedNumber'] = dn

sip = []

for i in df['SIP']:
    try:
        i = int(i)
        
        sip.append(i)
    except:
        sip.append('')
        

df['SIP'] = sip

df['AgentID'].replace('nan', '', inplace=True)
df['AgentName'].replace('nan', '', inplace=True)
df['HangupBy'].replace('nan', '', inplace=True)

c = []
for i in df['DialedNumber']:
    if i >= 400000000 and i <=500000000:
        c.append(0.031)
    else:
        c.append(0.0118)
        
df['CallRate'] = c


In [3]:
df.head()

,Campaign,LeadID,DIDNumber,DialedNumber,CallType,Date/Time,SIP,AgentID,AgentName,RingTime,BillSec,TotalTalktime,DialerDisposition,HangupCode,HangupDescription,HangupBy,CallRate
0,MSMEDIAAUS,WA_MOBILE_50K1,,419968772,OB,2020-03-11 15:32:09,,NaN,NaN,0.067,0.033,0.100,Answering Machine,16,Normal Clearing,NaN,0.031
1,MSMEDIAAUS,WA_MOBILE_50K1,,414900690,OB,2020-03-11 15:32:09,,NaN,NaN,0.550,0.050,0.600,Answering Machine,16,Normal Clearing,NaN,0.031
2,MSMEDIAAUS,WA_MOBILE_50K1,480034346,418949644,OB,2020-03-11 15:32:09,,NaN,NaN,0.383,0.800,1.183,Agent Busy - Maximum Wait Time,16,Normal Clearing,NaN,0.031
3,MSMEDIAAUS,WA_MOBILE_50K1,480034347,448892224,OB,2020-03-11 15:31:48,,NaN,NaN,0.000,0.000,0.000,Ringing - No Answer,16,Normal Clearing,NaN,0.031
4,MSMEDIAAUS,WA_MOBILE_50K1,480034346,404757860,OB,2020-03-11 15:31:33,,NaN,NaN,0.000,0.000,0.000,Ringing - No Answer,16,Normal Clearing,NaN,0.031


# Import Disposition Report Data

In [4]:
df1 = pd.read_excel('disposition.xlsx')
df1 = df1.iloc[:,:18]

df1['Agent ID'] = df1['Agent ID'].str.split("(", expand=True)[0]

df1['Phone Number']= df1['Phone Number'].astype(str)
df1['Phone Number']= df1['Phone Number'].str.replace(" ()","")
df1['Phone Number'] = df1['Phone Number'].str.slice(2,11)

df1['DID Number']= df1['DID Number'].astype(str)
df1['DID Number']= df1['DID Number'].str.replace(" ()","")
df1['DID Number'] = df1['DID Number'].str.slice(2,11)

df1['Start Time']= pd.to_datetime(df1['Start Time'])
df1['End Time']= pd.to_datetime(df1['End Time'])

cht = []
for i in df1['Call Handling Time']:
    try:
        i = str(i)
        i = i.replace(":", "")
        h = i[:2]
        m = i[2:4]
        s = i[4:]
        h = int(h)
        m = int(m)
        s = int(s)
        min = (h*60)+(m*1)+(s/60)
        min = round(min,3)
        cht.append(min)
    except:
        cht.append(0)

df1['Call Handling Time'] = cht

tt = []
for i in df1['Talk Time']:
    try:
        i = str(i)
        i = i.replace(":", "")
        h = i[:2]
        m = i[2:4]
        s = i[4:]
        h = int(h)
        m = int(m)
        s = int(s)
        min = (h*60)+(m*1)+(s/60)
        min = round(min,3)
        tt.append(min)
    except:
        tt.append(0)

df1['Talk Time'] = tt

wd = []
for i in df1['Wrapup Duration']:
    try:
        i = str(i)
        i = i.replace(":", "")
        h = i[:2]
        m = i[2:4]
        s = i[4:]
        h = int(h)
        m = int(m)
        s = int(s)
        min = (h*60)+(m*1)+(s/60)
        min = round(min,3)
        wd.append(min)
    except:
        wd.append(0)

df1['Wrapup Duration'] = wd

df1.columns = df1.columns.str.replace(' ', '')
df1.rename(columns={'PhoneNumber': 'DialedNumber', 'Lead':'LeadID','Sip':'SIP','Disposition':'AgentDisposition' }
           ,inplace = True)


k = []

for i in df1['CallHandlingTime']:
    if i <=0.50:
        k.append('Worst')
    elif i > 0.50 and i <=1:
         k.append('Poor')
    elif i > 1 and i <=3:
         k.append('Fair')
    elif i > 3 and i <=5:
         k.append('Good')
    else:
        k.append('Excellent')
        
df1['CallQuality'] = k

df1["AgentID"]= df1["AgentID"].str.replace('user', "User ", case= False)
df1.drop('Prefix', axis=1, inplace=True)

In [5]:
df1.head()

,Campaign,LeadID,AgentID,AgentName,SIP,DialedNumber,DIDNumber,CallType,StartTime,EndTime,CallHandlingTime,TalkTime,WrapupDuration,AgentDisposition,SubDisposition,Remarks,Review,CallQuality
0,MSMEDIAAUS,WA_Mobile_50K1,User 9,Sonia,60528,423369682,480034347,OB,2020-03-11 15:31:59,2020-03-11 15:32:10,0.183,0.167,0.017,ANSWERING MACHINE,NO SUBDISPOSITION,NaN,CALL,Worst
1,MSMEDIAAUS,WA_Mobile_50K1,User 2,Daniel,60526,417935335,480034346,OB,2020-03-11 15:31:48,2020-03-11 15:32:21,0.550,0.333,0.217,NI,NO SUBDISPOSITION,NaN,CALL,Poor
2,MSMEDIAAUS,WA_Mobile_50K1,User 9,Sonia,60528,438505404,480034347,OB,2020-03-11 15:31:32,2020-03-11 15:31:57,0.417,0.333,0.083,DNC,NO SUBDISPOSITION,NaN,CALL,Worst
3,MSMEDIAAUS,WA_Mobile_50K1,User 2,Daniel,60526,400581298,480034346,OB,2020-03-11 15:31:21,2020-03-11 15:31:45,0.400,0.400,0.000,NO ACCIDENT,NO SUBDISPOSITION,NaN,CALL,Worst
4,MSMEDIAAUS,WA_Mobile_50K1,User 14,Clinton,60541,419048769,480034346,OB,2020-03-11 15:31:10,2020-03-11 15:32:07,0.950,0.933,0.017,NI,NO SUBDISPOSITION,NaN,CALL,Poor


# Import CDR Bill Data

In [6]:
df6 = pd.read_csv('20200311.xls','\t')
df6.rename(columns={'callstart': 'CallStart','Amt':'CallCost','DialedNo':'DialedNumber','CallerID':'DIDNumber'}, 
           inplace= True)
df6.drop('CallerIP', axis=1, inplace= True)

df6['DialedNumber']= df6['DialedNumber'].astype(str)
aus_bill = df6[df6.DialedNumber.str.startswith('6')]
can_bill = df6[~df6.DialedNumber.str.startswith('6')]

aus_bill = pd.DataFrame(aus_bill)
can_bill = pd.DataFrame(can_bill)

aus_bill['DIDNumber']    = aus_bill['DIDNumber'].astype(str)
aus_bill['DIDNumber']    = aus_bill['DIDNumber'].str.replace(" ()","")
aus_bill['DIDNumber']    = aus_bill['DIDNumber'].str.slice(2,11)
aus_bill['DialedNumber'] = aus_bill['DialedNumber'].str.replace(" ()","")
aus_bill['DialedNumber'] = aus_bill['DialedNumber'].str.slice(2,11)

aus_bill['DIDNumber']    = aus_bill['DIDNumber'].astype(int)
aus_bill['DialedNumber'] = aus_bill['DialedNumber'].astype(int)
aus_bill['CallStart']    = pd.to_datetime(aus_bill['CallStart'])

In [7]:
aus_bill.head()

,DialedNumber,Duration,DIDNumber,CallStart,CallCost
0,415096670,6,480034347,2020-03-11 01:16:51,0.0031
1,413543699,6,480034346,2020-03-11 01:17:07,0.0031
2,418886426,6,480034347,2020-03-11 01:17:10,0.0031
3,435245245,6,480034347,2020-03-11 01:17:23,0.0031
4,421822212,6,480034347,2020-03-11 01:17:25,0.0031
